<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/backtesting_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtesting

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_price(self, start:int='20180501', interval='day'):
    url ='https://api.finance.naver.com/siseJson.naver?symbol=' + self.code + ' &requestType=1&startTime=' \
    + start + '&endTime=' + str(datetime.datetime.now().date().strftime("%Y%m%d")) +  '&timeframe=' + interval

    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], int), 'High':np.array(data[:,2], int), 'Low':np.array(data[:,3], int),\
       'Close':np.array(data[:,4], int), 'Volume':np.array(data[:,5], int)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    return df

In [25]:
kospi = Stock('KOSPI')
kospi = kospi.get_price('20000101')
for i in kospi.columns[:-1].values:
  kospi[i] = [x/100 for x in kospi[i]]

In [26]:
kosdaq = Stock('KOSDAQ')
kosdaq = kosdaq.get_price('20000101')
for i in kosdaq.columns[:-1].values:
  kosdaq[i] = [x/100 for x in kosdaq[i]]

In [41]:
kosdaq[300:]

Open     High      Low    Close   Volume
2001-03-30   693.30   698.20   667.80   684.30   292828
2001-04-02   684.80   689.60   679.60   682.90   257970
2001-04-03   666.10   666.60   655.00   662.40   289320
2001-04-04   643.70   659.50   641.80   643.40   255877
2001-04-06   676.00   677.70   662.10   665.10   256744
...             ...      ...      ...      ...      ...
2021-08-30  1030.65  1035.77  1027.84  1031.84  1178120
2021-08-31  1032.77  1038.33  1026.36  1038.33  1243473
2021-09-01  1038.09  1046.00  1035.09  1045.98  1145478
2021-09-02  1048.15  1048.20  1042.36  1047.19  1292476
2021-09-03  1049.81  1054.53  1047.12  1053.85  1411177

[5053 rows x 5 columns]

In [59]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma3 = self.I(SMA, price, 3)
        self.ma5 = self.I(SMA, price, 5)
        self.ma10 = self.I(SMA, price, 10)


    def next(self):
        if crossover(self.data.Close, self.ma3) and crossover(self.data.Close, self.ma5) and crossover(self.data.Close, self.ma10):
            self.buy()
        elif crossover(self.ma3, self.data.Close) and crossover(self.ma5, self.data.Close) and crossover(self.ma10, self.data.Close):
            self.sell()


asset = kosdaq[1000:]
bt = Backtest(asset, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()

In [60]:
print(stats)

Start                     2004-02-06 00:00:00
End                       2021-09-03 00:00:00
Duration                   6419 days 00:00:00
Exposure Time [%]                     99.2878
Equity Final [$]                        21601
Equity Peak [$]                       43607.6
Return [%]                             116.01
Buy & Hold Return [%]                 140.979
Return (Ann.) [%]                     4.55941
Volatility (Ann.) [%]                 22.6916
Sharpe Ratio                         0.200929
Sortino Ratio                        0.322196
Calmar Ratio                        0.0623283
Max. Drawdown [%]                    -73.1514
Avg. Drawdown [%]                    -6.01514
Max. Drawdown Duration     4694 days 00:00:00
Avg. Drawdown Duration      125 days 00:00:00
# Trades                                  207
Win Rate [%]                          45.4106
Best Trade [%]                        30.4249
Worst Trade [%]                      -41.7318
Avg. Trade [%]                    

In [61]:
bt.plot()

Row(id='8733', ...)